In [1]:
import cv2
import numpy as np
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
from data import common

In [2]:
def imshows(ims, figsize=None, titles=[]):
    figsize = figsize or (len(ims) * 6, 4)
    plt.figure(figsize=figsize)
    for i, im in enumerate(ims):
        plt.subplot(1, len(ims), i + 1)
        plt.imshow(im)
        if len(titles) > i:
            plt.title(titles[i])
    plt.tight_layout()
    plt.show()

In [3]:
base = Path("dataset/benchmark")

In [4]:
list(base.glob("*"))

[WindowsPath('dataset/benchmark/BSD100'),
 WindowsPath('dataset/benchmark/Manga109'),
 WindowsPath('dataset/benchmark/Set14'),
 WindowsPath('dataset/benchmark/Set5'),
 WindowsPath('dataset/benchmark/Urban100')]

First, we need to crop the HR image into the size having a common multiplier of 2, 3, 4. If you don't adjust HR into this way it is hard to make pixel to pixel matching of HR-LR pair for all resized images of x2, x3, x4.

In [5]:
for data in base.glob("*"):

    save_path = data / "HR"
    save_path.mkdir(exist_ok=True)


    for p in data.glob("*"):
        try:
            img = cv2.imread(str(p))
            h, w, c = img.shape

            ix, iy = int(h/12) * 6, int(w/12) * 6
            cx, cy = int(h/2), int(w/2)
            img = img[cx - ix:cx + ix, cy - iy:cy + iy, :]

            cv2.imwrite(str(save_path / p.name), img)
        except Exception as e:
            print(data.name, p.name, e)
        finally:
            pass

BSD100 HR 'NoneType' object has no attribute 'shape'
Manga109 HR 'NoneType' object has no attribute 'shape'
Set14 HR 'NoneType' object has no attribute 'shape'
Set5 HR 'NoneType' object has no attribute 'shape'
Urban100 HR 'NoneType' object has no attribute 'shape'


In [17]:
for data in (base).glob("*"):
    
    source_path = data / "HR"
    save_path = data / "LR_bicubic"
    save_path.mkdir(exist_ok=True)


    for p in source_path.glob("*"):
        try:
            img = cv2.imread(str(p))
            h, w, c = img.shape

            for scale in [2, 3, 4]:
                ch, cw = int(h/scale), int(w/scale)
                assert ch*scale == h and ch*scale == h, "Scale error!"
                img = cv2.resize(img, dsize=(cw, ch), interpolation=cv2.INTER_CUBIC)

                target_path = save_path / "x{:1d}".format(scale)
                target_path.mkdir(exist_ok=True)
                name = target_path / "{}x{:1d}{}".format(p.stem, scale, p.suffix)
                cv2.imwrite(str(name), img)
        except Exception as e:
            print(data.name, p.name, e)
        finally:
            pass

Let's check benchmark dataset's rgb_range. It's is helpful for dataloader setting

In [16]:
for data in base.glob("*"):
    path = list((data / "HR").glob("*"))[0]
    img = cv2.imread(str(path))
    rgb_range = 255 if img.max() > 230 else 1
    print("{:10s} has {} color channel and rgb range max is {}. File extention is {}.".format(data.name, img.shape[-1], rgb_range, path.suffix))

BSD100     has 3 color channel and rgb range max is 255. File extention is .png.
Manga109   has 3 color channel and rgb range max is 255. File extention is .png.
Set14      has 3 color channel and rgb range max is 255. File extention is .png.
Set5       has 3 color channel and rgb range max is 255. File extention is .png.
Urban100   has 3 color channel and rgb range max is 255. File extention is .png.


Following lines for BSD500 data prepare

In [18]:
base = Path("dataset/BSD500")
base.exists()

True

In [19]:
for data in base.glob("*"):

    save_path = data / "HR"
    save_path.mkdir(exist_ok=True)


    for p in data.glob("*.JPG"):
        try:
            img = cv2.imread(str(p))
            h, w, c = img.shape

            ix, iy = int(h/12) * 6, int(w/12) * 6
            cx, cy = int(h/2), int(w/2)
            img = img[cx - ix:cx + ix, cy - iy:cy + iy, :]

            cv2.imwrite(str(save_path / p.name), img)
        except Exception as e:
            print(data.name, p.name, e)
        finally:
            pass

In [20]:
for data in base.glob("*"):
    
    source_path = data / "HR"
    save_path = data / "LR_bicubic"
    save_path.mkdir(exist_ok=True)


    for p in source_path.glob("*.JPG"):
        try:
            img = cv2.imread(str(p))
            h, w, c = img.shape

            for scale in [2, 3, 4]:
                ch, cw = int(h/scale), int(w/scale)
                assert ch*scale == h and ch*scale == h, "Scale error!"
                img = cv2.resize(img, dsize=(cw, ch), interpolation=cv2.INTER_CUBIC)

                target_path = save_path / "x{:1d}".format(scale)
                target_path.mkdir(exist_ok=True)
                name = target_path / "{}x{:1d}{}".format(p.stem, scale, p.suffix)
                cv2.imwrite(str(name), img)
        except Exception as e:
            print(data.name, p.name, e)
        finally:
            pass